# This Capstone project is the final assignment of the IBM data science professional certificate #

### In 2019, what is the most strategical location to open a fine dining Italian restaurant in Melbourne, Australia? ###

**Part 1 : First, we want to create a database that encapsulates the main districts in the city center of Melbourne with their latitude and longitude**

In [1]:
#Step 1: Importing the libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import webbrowser

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')





Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.88 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.77 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  34.81 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  41.21 MB/s
Libraries imported.


In [2]:
#Step 2: Let's visualize the area we will be working with

print ('We will focus on the central districts of Melbourne that can be visualized at the following wikipedia page: https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs')
print ('See on the page: Inner City municipalities and their suburbs')

We will focus on the central districts of Melbourne that can be visualized at the following wikipedia page: https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs
See on the page: Inner City municipalities and their suburbs


In [3]:
#step 3: Let's create a dataframe by selecting the 10 most central districts. We will call it 'mcc' for 'Melbourne city center'

mcc = pd.DataFrame({'District': ['CBD','East Melbourne','St Kilda Road','Southbank','Docklands','North Melbourne','Parkville','Carlton South','Fitzroy','South Yarra'],
                    'PC': ['3000','3002','3004','3006','3008','3051','3052','3053','3065','3141'],
                    'Latitude': [-37.814,-37.812,-37.84,-37.825,-37.818,-37.797,-37.786,-37.799,-37.80,-37.839],
                    'Longitude': [144.961,144.984,144.98,144.964,144.936,144.94,144.952,144.967,144.98,144.991]},
                      index = ['0', '1', '2','3','4','5','6','7','8','9'])
mcc

,District,Latitude,Longitude,PC
0,CBD,-37.814,144.961,3000
1,East Melbourne,-37.812,144.984,3002
2,St Kilda Road,-37.840,144.980,3004
3,Southbank,-37.825,144.964,3006
4,Docklands,-37.818,144.936,3008
5,North Melbourne,-37.797,144.940,3051
6,Parkville,-37.786,144.952,3052
7,Carlton South,-37.799,144.967,3053
8,Fitzroy,-37.800,144.980,3065
9,South Yarra,-37.839,144.991,3141


In [4]:
#step 4: Now we create a map of Melbourne with our disctrics centered on the latitude and longitude of Southbank to have a first geographical snapshot

latitude = -37.82
longitude = 144.97

map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(mcc['Latitude'], mcc['Longitude'], mcc['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

As you can see on the map, the blue dots are organized around the cbd which is a small rectangle in the center of the city

**Part 2 : Now let's analyze our data using Foursquare API in order to find the best location for our restaurant**

**2.a : Let's first look at the 10 districts and understand the popular venues within them**

In [5]:
#step 5: Let's first log into Foursquare:

CLIENT_ID = 'JEZBBNGURDWQZ5SLJXLENN3DV51O3VHGAIUWYP3BWKUJMAPX' # your Foursquare ID
CLIENT_SECRET = 'F4BLEVSS3TVDRDMETC4X4UHA4NUJJJLP0ISYNP5QEZF5BXAQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
            
 

Your credentails:
CLIENT_ID: JEZBBNGURDWQZ5SLJXLENN3DV51O3VHGAIUWYP3BWKUJMAPX
CLIENT_SECRET:F4BLEVSS3TVDRDMETC4X4UHA4NUJJJLP0ISYNP5QEZF5BXAQ


In [6]:
#step 6: We create a function that will help retrieve the venues in the different distritcs

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,     
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#step 7: We now create the dataframe associated with these venues

mcc_venues = getNearbyVenues(names=mcc['District'],
                                   latitudes=mcc['Latitude'],
                                   longitudes=mcc['Longitude']
                                  )

CBD
East Melbourne
St Kilda Road
Southbank
Docklands
North Melbourne
Parkville
Carlton South
Fitzroy
South Yarra


In [8]:
#step 8: Check size and take a look at the newly create dataframe

print(mcc_venues.shape)
mcc_venues.head()

(482, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CBD,-37.814,144.961,Tipo 00,-37.813527,144.961978,Italian Restaurant
1,CBD,-37.814,144.961,Brother Baba Budan,-37.813445,144.962137,Coffee Shop
2,CBD,-37.814,144.961,Kirk's Wine Bar,-37.813661,144.961351,Wine Bar
3,CBD,-37.814,144.961,Manchester Press,-37.813740,144.962132,Café
4,CBD,-37.814,144.961,Red Spice Road,-37.815332,144.961644,Thai Restaurant


In [9]:
#step 9: Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(mcc_venues['Venue Category'].unique())))

There are 133 uniques categories.


In [10]:
#step 10: Let's group the venues by districts so we understand which are popular

# one hot encoding
mcc_onehot = pd.get_dummies(mcc_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
mcc_onehot['District'] = mcc_venues['District'] 

# move district column to the first column
fixed_columns = [mcc_onehot.columns[-1]] + list(mcc_onehot.columns[:-1])
mcc_onehot = mcc_onehot[fixed_columns]

#group by ditrict by taking the mean of occurence of each category
mcc_grouped = mcc_onehot.groupby('District').mean().reset_index()
mcc_grouped.head()



,District,African Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community Center,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Truck,Football Stadium,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Nightclub,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Social Club,Souvlaki Shop,Spa,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yunnan Restaurant,Zoo,Zoo Exhibit
0,CBD,0.00,0.01,0.01,0.01,0.010000,0.000000,0.00,0.01,0.000000,0.060000,0.0,0.00,0.01,0.000000,0.01,0.01,0.0,0.01,0.030000,0.0,0.00,0.120000,0.0,0.000000,0.000000,0.02,0.02,0.03,0.070000,0.01,0.02,0.0,0.000000,0.01,0.000000,0.050000,0.01,0.0,0.01,0.01,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.01,0.01,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.040000,0.03,0.01,0.01,0.03,0.01,0.000000,0.0,0.0,0.010000,0.000000,0.00,0.000000,0.01,0.02,0.0,0.0,0.00,0.01,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.0,0.000000,0.0,0.000000,0.01,0.00,0.01,0.00,0.00,0.0,0.01,0.0,0.000000,0.000000,0.03,0.01,0.01,0.00,0.01,0.0,0.0,0.0,0.020000,0.0,0.00,0.01,0.01,0.020000,0.01,0.02,0.0,0.000000,0.0,0.000000,0.00,0.010000,0.000000,0.000000,0.0,0.0
1,Carlton South,0.00,0.00,0.00,0.00,0.022222,0.000000,0.00,0.00,0.022222,0.044444,0.0,0.00,0.00,0.022222,0.00,0.00,0.0,0.00,0.022222,0.0,0.00,0.111111,0.0,0.022222,0.022222,0.00,0.00,0.00,0.133333,0.00,0.00,0.0,0.000000,0.00,0.044444,0.022222,0.00,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.0,0.00,0.022222,0.00,0.00,0.000000,0.00,0.00,0.0,0.044444,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.044444,0.022222,0.022222,0.022222,0.00,0.111111,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,0.022222,0.022222,0.00,0.022222,0.00,0.00,0.0,0.0,0.00,0.00,0.022222,0.000000,0.0,0.000000,0.000000,0.022222,0.0,0.000000,0.0,0.022222,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.000000,0.0,0.022222,0.00,0.044444,0.000000,0.022222,0.0,0.0
2,Docklands,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.090909,0.00,0.00,0.00,0.000000,0.00,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000,0.090909,0.00,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.090909,0.0,0.00,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00

In [11]:
#step 11: Let's print each district along with the top 5 most common venues
num_top_venues = 5

for hood in mcc_grouped['District']:
    print("----"+hood+"----")
    temp = mcc_grouped[mcc_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CBD----
                venue  freq
0                Café  0.12
1         Coffee Shop  0.07
2                 Bar  0.06
3        Dessert Shop  0.05
4  Italian Restaurant  0.04


----Carlton South----
                venue  freq
0         Coffee Shop  0.13
1                Café  0.11
2  Italian Restaurant  0.11
3        Gourmet Shop  0.04
4            Wine Bar  0.04


----Docklands----
                       venue  freq
0           Sushi Restaurant  0.09
1  Middle Eastern Restaurant  0.09
2                       Park  0.09
3      Performing Arts Venue  0.09
4                      Plaza  0.09


----East Melbourne----
              venue  freq
0              Café  0.15
1             Hotel  0.09
2              Park  0.09
3  Sculpture Garden  0.06
4       Coffee Shop  0.06


----Fitzroy----
                           venue  freq
0                           Café  0.15
1                            Bar  0.10
2                   Cocktail Bar  0.06
3                            Pub  0.05
4  V

Taking a look at these data we can see that Italian restaurants are among the most popular venues in Carlton district with a frequency rate of 0.12. They are also the 2nd most popular venue in South Yarra with a rate of 0.11. Last but not least, the CBD has a frequency rate of 0.04 but given the density of venues in that area it should also be considered in our analysis. Cafe/Coffee shop are the most popular venues by far in almost every district which tends to confirm the reputation of Melbourne for its cafes. Having a good italian coffee on the menu will probably adds to the value of a future italian restaurant in the city.

**2.b : Let's now consider the 3 districts where customers are likely to go for italian food. It would be interesting to get a list of the italian restaurants in these districts**

In [12]:
#Step 12: Return a list of Italian restaurants by district

#Define a search query

print ('South Yarra:')

latitude1 = -37.839 #South Yarra latitude
longitude1 = 144.991 #South Yarra Longitude
search_query = 'Italian'
radius = 500

#Define the corresponding URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius, LIMIT)

#Send the GET Request to obtain the results

results1 = requests.get(url).json()

# assign relevant part of JSON to venues

venues1 = results1['response']['venues']

# tranform venues into a dataframe 

yarra_it = json_normalize(venues1)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in yarra_it.columns if col.startswith('location.')] + ['id']
yarra_it_filtered = yarra_it.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
yarra_it_filtered['categories'] = yarra_it_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
yarra_it_filtered.columns = [column.split('.')[-1] for column in yarra_it_filtered.columns]

yarra_it_filtered




South Yarra:


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Cucinetta,Italian Restaurant,4/3 Murphy St,AU,South Yarra,Australia,103,"[4/3 Murphy St, South Yarra VIC 3141, Australia]","[{'lng': 144.990213, 'lat': -37.838314, 'label...",-37.838314,144.990213,South Yarra,3141,VIC,51c506d1498e8f1d0cf01993


In [13]:
#Repeat same analysis with Carlton

print ('Carlton:')

latitude2 = -37.799 #Carlton latitude
longitude2 = 144.967 #Carlton Longitude
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query, radius, LIMIT)
results2 = requests.get(url).json()
venues2 = results2['response']['venues']
carlton_it = json_normalize(venues2)
filtered_columns = ['name', 'categories'] + [col for col in carlton_it.columns if col.startswith('location.')] + ['id']
carlton_it_filtered = carlton_it.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
carlton_it_filtered['categories'] = carlton_it_filtered.apply(get_category_type, axis=1)
carlton_it_filtered.columns = [column.split('.')[-1] for column in carlton_it_filtered.columns]
carlton_it_filtered

Carlton:


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Universal Italian Restaurant,Italian Restaurant,139-141 Lygon St,AU,Carlton,Australia,NaN,542,"[139-141 Lygon St, Carlton VIC 3053, Australia]","[{'lng': 144.9663526478978, 'lat': -37.8038504...",-37.803850,144.966353,NaN,3053,VIC,4c1c88a1b9f876b00b197c46
1,Italian Chamber Of Commerce and Industry in Au...,Office,"Level 1, 185 Faraday Street",AU,Carlton,Australia,NaN,122,"[Level 1, 185 Faraday Street, Carlton VIC 3054...","[{'lng': 144.968387, 'lat': -37.798917, 'label...",-37.798917,144.968387,NaN,3054,VIC,4e9b93de7ee539c857a2b72e
2,Gemma Simply Italian,Italian Restaurant,183 Elgin St,AU,Carlton,Australia,NaN,222,"[183 Elgin St, Carlton VIC 3053, Australia]","[{'lng': 144.96835382776104, 'lat': -37.797308...",-37.797309,144.968354,Carlton,3053,VIC,53524541498ee975a2fc5b86
3,Dolomiti Italian Style,Bike Shop,Elgin Street,AU,Carlton,Australia,Lygon St,184,"[Elgin Street (Lygon St), Carlton VIC 3053, Au...","[{'lng': 144.9678010646946, 'lat': -37.7974721...",-37.797472,144.967801,NaN,3053,VIC,4ee6946a9a52c1dadc4ab650
4,Centre of Italian studies,Community College,247 Cardigan st,AU,Carlton,Australia,NaN,278,"[247 Cardigan st, Carlton VIC, Australia]","[{'lng': 144.96520161165256, 'lat': -37.801064...",-37.801064,144.965202,NaN,NaN,VIC,4c5936602091a593934f5cd0
5,Museo Italiano,History Museum,199 Faraday St,AU,Carlton,Australia,lygon st,85,"[199 Faraday St (lygon st), Carlton VIC 3053, ...","[{'lng': 144.96794954217452, 'lat': -37.798819...",-37.798819,144.967950,NaN,3053,VIC,4d9a94cc9298b1f7eae84438
6,Bentornato Al Bar Italiano,Italian Restaurant,326 Lygon Street,AU,NaN,Australia,NaN,397,"[326 Lygon Street, Australia]","[{'lng': 144.964167, 'lat': -37.801778, 'label...",-37.801778,144.964167,NaN,NaN,NaN,504048b8e4b06a01b98f19dc


In [14]:
#Repeat same analysis with CBD

print ('CBD:')

latitude3 = -37.814 #CBD latitude
longitude3 = 144.961 #CBD Longitude
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude3, longitude3, VERSION, search_query, radius, LIMIT)
results3 = requests.get(url).json()
venues3 = results3['response']['venues']
cbd_it = json_normalize(venues3)
filtered_columns = ['name', 'categories'] + [col for col in cbd_it.columns if col.startswith('location.')] + ['id']
cbd_it_filtered = cbd_it.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
cbd_it_filtered['categories'] = cbd_it_filtered.apply(get_category_type, axis=1)
cbd_it_filtered.columns = [column.split('.')[-1] for column in cbd_it_filtered.columns]
cbd_it_filtered

CBD:


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Lil Solito Posto Italian,Italian Restaurant,Collins Street,AU,Melbourne,Australia,NaN,38,"[Collins Street, Melbourne VIC, Australia]","[{'lng': 144.96143363448942, 'lat': -37.813979...",-37.813980,144.961434,NaN,NaN,VIC,52f5f7d6498ed711dca591b3
1,Little Avenue Des Italians Espresso Bar,Café,428 Little Bourke Street,AU,Melbourne,Australia,NaN,105,"[428 Little Bourke Street, Melbourne VIC 3000,...","[{'lng': 144.9598, 'lat': -37.81399, 'label': ...",-37.813990,144.959800,NaN,3000,VIC,582e3ba67ff1e43c193eb43b
2,Il Tempo,Italian Restaurant,8 Degraves St,AU,Melbourne,Australia,Flinders St,581,"[8 Degraves St (Flinders St), Melbourne VIC 30...","[{'lng': 144.96593122744065, 'lat': -37.817483...",-37.817484,144.965931,NaN,3000,VIC,4b4c3b12f964a520e7af26e3
3,Map Coffee,Coffee Shop,Elizabeth St,AU,Melbourne,Australia,NaN,373,"[Elizabeth St, Melbourne VIC 3000, Australia]","[{'lng': 144.96162723167922, 'lat': -37.810680...",-37.810681,144.961627,NaN,3000,VIC,4d3ce4aea2e4b1f7fb8aed25
4,Maccaroni Trattoria Italiana,Italian Restaurant,10 Manchester Lane,AU,Melbourne,Australia,NaN,478,"[10 Manchester Lane, Melbourne VIC 3000, Austr...","[{'lng': 144.9658402356701, 'lat': -37.8159582...",-37.815958,144.965840,"Melbourne CBD, Melbourne, VIC",3000,VIC,4dc0b7b7c65b268b41cbcb5a
5,osteria italiana,Italian Restaurant,NaN,AU,NaN,Australia,NaN,473,[Australia],"[{'lng': 144.96583557128906, 'lat': -37.815864...",-37.815865,144.965836,NaN,NaN,NaN,538c413b498e7b711ddca34e


Having a look at these data, it is quite clear that Carlton and CBD propose the most important density of Italian restaurants in Melbourne city center. 
Both districts are also fairly close to Victoria market which is the biggest market in Melbourne and a great place to get fresh products

**2.c : Now let's have a look at some ratings and comments about the restaurants in Carlton and CBD and see if we can get some insight from it**

Let's first look at Universal Italian Restaurant in Carlton


In [15]:
#Step 13: We now explore the venues in more detail

#Get url

venue_id = '4c1c88a1b9f876b00b197c46' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

#Send GET request for result

result = requests.get(url).json()

#Get venue rating

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



8.3


That's a fairly good rating, let's see how it compares to the other restaurants

In [16]:

#Universal Italian Restaurant
print ('Carlton: Universal Italian Restaurant')
venue_id = '4c1c88a1b9f876b00b197c46' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

#Gemma Simply Italian
print ('Carlton: Gemma Simply Italian')
venue_id = '53524541498ee975a2fc5b86' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
#Bentornato Al Bar Italiano
print ('Carlton: Bentornato Al Bar Italiano')
venue_id = '504048b8e4b06a01b98f19dc' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
#Lil Solito Posto Italian
print ('CBD: Lil Solito Posto Italian')
venue_id = '52f5f7d6498ed711dca591b3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
#Il Tempo
print ('CBD: Il Tempo')
venue_id = '4b4c3b12f964a520e7af26e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
#Maccaroni Trattoria Italiana
print ('CBD: Maccaroni Trattoria Italiana')
venue_id = '4dc0b7b7c65b268b41cbcb5a' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
#osteria italiana
print ('CBD: osteria italiana')
venue_id = '538c413b498e7b711ddca34e' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



Carlton: Universal Italian Restaurant
8.3
Carlton: Gemma Simply Italian
7.8
Carlton: Bentornato Al Bar Italiano
This venue has not been rated yet.
CBD: Lil Solito Posto Italian
This venue has not been rated yet.
CBD: Il Tempo
7.0
CBD: Maccaroni Trattoria Italiana
7.6
CBD: osteria italiana
This venue has not been rated yet.


Looking at the ratings, it seems 'Universal Italian Restaurant' and 'Gemma Simply Italian' in Carlton, 'Maccaroni Trattoria Italiana' in CBD could be potential competitors. 
Let's have a look at some comments from the customers

In [17]:
#Step 14: Get some venue tips

## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips

print ('Carlton: Universal Italian Restaurant')
venue_id = '4c1c88a1b9f876b00b197c46'  

#Create url and get request

url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()

#Get tips

tips = results['response']['tips']['items']

#Format columns and display tips

pd.set_option('display.max_colwidth', -1)
carltonuni_tips = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
carltonuni_tips_filtered = carltonuni_tips.loc[:, filtered_columns]

# display tips
carltonuni_tips_filtered


Carlton: Universal Italian Restaurant


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Universal has some of Melbourne's biggest, cheapest and best Italian dishes you can find. It's by far the best value for money place I have found that puts a smile on my face everytime!",5,0,52b1577f498efe9d3286fef5,Richard,Czerwik,male,44701384
1,Chicken schnitzel at $14 is huge - enough for 2 pax. Seafood Pasta marinara (entree is enough. But can order main portion if big eater) is good 2. These 2 dishes can be shared by 3 pax,2,0,5b5da0412f97ec002cf12c86,pin,lee,female,4621745


In [18]:
print ('Carlton: Gemma Simply Italian')
venue_id = '53524541498ee975a2fc5b86'  

url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()
tips = results['response']['tips']['items']
pd.set_option('display.max_colwidth', -1)
carltongemma_tips = json_normalize(tips) # json normalize tips
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
carltongemma_tips_filtered = carltongemma_tips.loc[:, filtered_columns]
carltongemma_tips_filtered

Carlton: Gemma Simply Italian


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Great fresh pasta and gnocchi. Friendly service. Good buzz.,0,0,569a2e28498ec7b137e832b4,Lindsay,R,male,34236212
1,"Loved this place, great old charm about the place, fantastic gnocchi, will be back without a doubt.",0,0,5a2876d68b98fd31f8d9264d,Monica,Fulco,female,460006895


In [19]:
print ('CBD: Maccaroni Trattoria Italiana')
venue_id = '4dc0b7b7c65b268b41cbcb5a'  

url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()
tips = results['response']['tips']['items']
pd.set_option('display.max_colwidth', -1)
cbdmac_tips = json_normalize(tips) # json normalize tips
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
cbdmac_tips_filtered = cbdmac_tips.loc[:, filtered_columns]
cbdmac_tips_filtered

CBD: Maccaroni Trattoria Italiana


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Ossobuco is just the right meal if you're looking to load up on a weighty portion of protein and carbs. Lamb was perfectly cooked and very tasty. I'll visit again. Cappuccino was perfect also.,2,0,5191e414498e59ff672528d3,Wilhelm,Engelbrecht,male,27751025
1,"Authentic & delish!!! Gnocci, Lasagne & Chicken Soup (sublime) - excellent staff, friendly & accommodating. Cakes are must. Great to be back...",1,0,56146a71498e3b864e01f96b,Kim,Barry,female,184766


In [20]:
print ('CBD: Il Tempo')
venue_id = '4b4c3b12f964a520e7af26e3'  

url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
results = requests.get(url).json()
tips = results['response']['tips']['items']
pd.set_option('display.max_colwidth', -1)
cbdtempo_tips = json_normalize(tips) # json normalize tips
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
cbdtempo_tips_filtered = cbdtempo_tips.loc[:, filtered_columns]
cbdtempo_tips_filtered

CBD: Il Tempo


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Ricotta and spinach ravioli with toungue and pumpkin puree was sublime! Everything was super flavorsome and balanced. I'm in love with this place.,2,0,540e6d58498ef052210fa3cc,Cara,Libro,male,94386403
1,Awesome restaurant in a cool street. The pizza is just perfect! I had a white pizza with chicken and taste was absolutely delicious. The pasta was also perfect. Staff was helpful and very friendly.,1,0,5677313a498e55d82a33f18e,Tord,S,male,139651651


From what we can gather it seems all comments are pretty positive. Gnocchi is mentioned 3 times which might indicate that this dish is cherished by Melbournians.
Size of portions, quality of service, preparation of coffee are also underlined so those areas might be worth considering when opening a restaurant.
For the rest, it seems both Carlton & CBD could be good locations for a future restaurant. The number of restaurants in each area is somehow identical. 
The proximity of the Italian Chamber of commerce and Industry, the Centre of Italian studies and the Museo Italiano listed among the Carlton venues indicate this area as being the Italian heart of the city.
As a consequence, the proportion of italian citizens or people from Italian origin is probably more important in that area.
A more refined or authentic cuisine could then be very much appreciated and that would perfectly correspond to the philosophy of the restaurant that we would like to start.

**All in all, the CBD is the densest and most touristic area in Melbourne while Carlton is the Italian core of the city making both districts a perfect location to start a fine dining Italian restaurant in Melbourne**